In [1]:
import fastf1 as ff1
from fastf1 import plotting
from fastf1.core import Laps

import pandas as pd
import numpy as np

In [2]:
# Enable the cache by providing the name of the cache folder, speed up
ff1.Cache.enable_cache('cache')

# Setup plotting, setup the plot (bg: black, ...)
plotting.setup_mpl()

ff1.Cache.offline_mode(True)

In [87]:
FILE_CIRCUITS = 'f1_unique_circuits_complete.csv'

circuit_info = pd.read_csv(FILE_CIRCUITS, delimiter=",")

In [3]:
session = ff1.get_session(2025, 'Monza', 'Q')
session.load()

core           INFO 	Loading data for Italian Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '44', '63', '12', '5', '14', '22', '87', '27', '55', '23', '31', '6', '18', '43', '10', '30']


In [4]:
driver = 'VER'
# fastest lap of the driver
fastest_lap = session.laps.pick_drivers(driver).pick_fastest()
telemetry_driver = fastest_lap.get_telemetry().add_distance()


In [5]:
max(telemetry_driver['Speed'])

348.0

In [6]:
def calculate_physics(physics_df):
    # Convert speed from km/h to m/s and add it as a new column
    physics_df['v_m/s'] = physics_df['Speed'] / 3.6

    # Convert Time to seconds (float) and add it as a new column
    physics_df['time_s'] = physics_df['Time'] / np.timedelta64(1, 's')

    # Calculate longitudinal (forward/backward) acceleration (ax)
    # This is the derivative of velocity (v) with respect to time (t), or a = dv/dt
    # np.gradient() is a standard way to compute this from arrays of data.
    physics_df['ax_m/s^2'] = np.gradient(physics_df['v_m/s']) / np.gradient(physics_df['time_s'])

    # Smooth the acceleration data.
    # Telemetry data is often "noisy" (spiky), which makes derivatives very unstable.
    # np.convolve() applies a simple 3-point moving average to smooth out the spikes,
    # giving a more realistic acceleration value.
    physics_df['ax_smooth_m/s^2'] = np.convolve(
        physics_df['ax_m/s^2'], 
        np.ones((3,))/3, 
        mode='same'
    )

    # --- MODEL I: STATIC WEIGHT ONLY ---
    m = 798    # kg (Mass)
    g = 9.81   # m/s^2 (Gravity)
    staticWeightTot = m * g
    fractionWeightFront = 0.46

    # Calculate the static weight (in Newtons) on the front and rear axles
    staticWeightFront = staticWeightTot * fractionWeightFront
    staticWeightRear = staticWeightTot - staticWeightFront

    # Add the static loads to the DataFrame
    physics_df['loadFront_static_N'] = staticWeightFront
    physics_df['loadRear_static_N'] = staticWeightRear

    # --- MODEL II: INERTIAL LOAD TRANSFER ADDED ---
    CoGheight = 0.25    # m (Center of Gravity height)
    wheelbase = 3.6     # m (Wheelbase)

    # Calculate the *change* in load (deltaLoad) caused by longitudinal acceleration (ax)
    # This is the core "load transfer" equation.
    # If ax_smooth is positive (accelerating), deltaLoad becomes negative.
    physics_df['deltaLoad_Inertial_N'] = -CoGheight / wheelbase * m * physics_df['ax_smooth_m/s^2']

    # Add the inertial load to the static load
    # Calculate the new, dynamic load on the front and rear axles
    # Front Axle:
    #   - If accelerating (deltaLoad < 0), the front load *decreases*.
    physics_df['loadFront_Inertial_N'] = physics_df['loadFront_static_N'] + physics_df['deltaLoad_Inertial_N']
    # Rear Axle:
    #   - If accelerating (deltaLoad < 0), subtracting a negative *increases* the rear load.
    physics_df['loadRear_Inertial_N'] = physics_df['loadRear_static_N'] - physics_df['deltaLoad_Inertial_N']

    # --- MODEL III: AERODYNAMIC EFFECTS ADDED ---
    rho = 1.225         # kg/m^3 (Standard air density at sea level)
    CdA = 1.7           # Drag Coefficient * Frontal Area. Measures total air resistance.
    efficiency = 3.7      # Aerodynamic efficiency (Lift/Drag ratio). For F1, this is Downforce/Drag.
    ClA = efficiency*CdA  # Downforce Coefficient * Frontal Area. Measures total downforce.
    dragHeight = 0.5    # m (Height of drag force application)

    # Calculate Drag and Downforce (which depend on speed)
    # Calculate Drag Force (in Newtons)
    # Formula: Fd = 0.5 * rho * v^2 * CdA
    # Force increases with the *square* of velocity (v).
    physics_df['dragForce_N'] = 0.5 * CdA * rho * np.square(physics_df['v_m/s'])
    # Calculate Downforce (in Newtons)
    # Formula: Fl = 0.5 * rho * v^2 * ClA
    # This force also increases with the *square* of velocity.
    physics_df['downForce_N'] = 0.5 * ClA * rho * np.square(physics_df['v_m/s'])

    # Distribute the aerodynamic forces
    fractionDownforceFront = 0.4
    physics_df['downForceFront_N'] = physics_df['downForce_N'] * fractionDownforceFront
    physics_df['downForceRear_N'] = physics_df['downForce_N'] * (1 - fractionDownforceFront)

    # Calculate the load transfer caused *by drag*.
    # Drag pulls the car backward from a point *above* the ground (dragHeight),
    # which creates a moment that also shifts load to the rear axle.
    physics_df['deltaLoad_Drag_N'] = -dragHeight / wheelbase * physics_df['dragForce_N']

    # FINAL FRONT LOAD:
    # Model II load + downforce on the front + load transfer from drag
    physics_df['loadFront_AeroModel_N'] = (
        physics_df['loadFront_Inertial_N'] + 
        physics_df['downForceFront_N'] + 
        physics_df['deltaLoad_Drag_N']
    )
    # FINAL REAR LOAD:
    # Model II load + downforce on the rear - load transfer from drag
    physics_df['loadRear_AeroModel_N'] = (
        physics_df['loadRear_Inertial_N'] + 
        physics_df['downForceRear_N'] - 
        physics_df['deltaLoad_Drag_N']
    )

    return physics_df

# Longitudinal and Latitudinal acceleration

In [7]:
def convert_to_g(accelerations_ms2):
    g = 9.81
    return accelerations_ms2 / g


In [8]:
import numpy as np
import pandas as pd # Needed for type checking/conversion

def smooth_derivative(t_in, v_in, method="smooth"):
    """
    Computes a smooth estimation of a derivative (dv/dt) using a multi-point
    finite difference stencil to reduce noise amplification.
    """
    t = t_in.copy()
    v = v_in.copy()
    
    # (0) Prepare inputs: Ensure Time is transformed to seconds (a float array)
    try:
        # Check if t is a Pandas Series of timedelta objects
        if isinstance(t, pd.Series) and t.dtype == 'timedelta64[ns]':
            t = np.array([td.total_seconds() for td in t])
        else:
             # Assume it's a list/array of timedeltas needing conversion
             t = np.array([td.total_seconds() for td in t])
    except:
        # If it throws an error, it's likely already a float/int array (e.g., Distance)
        t = np.array(t)
        
    v = np.array(v)
    
    # Assert they have the same size and initialize output array
    assert t.size == v.size
    n = t.size
    dvdt = np.zeros(n)
    
    if n < 6:
        # Fallback to simple gradient if not enough points for the stencil
        return np.gradient(v, t)

    # (1) Manually compute points out of the stencil (boundary conditions)
    # The first few and last few points can't use the full stencil, so we use
    # a simpler, but stable, forward/backward difference approximation.
    
    dvdt[0] = (v[1]-v[0])/(t[1]-t[0])
    dvdt[1] = (v[2]-v[0])/(t[2]-t[0])
    dvdt[2] = (v[3]-v[1])/(t[3]-t[1])
    
    dvdt[n-1] = (v[n-1]-v[n-2])/(t[n-1]-t[n-2])
    dvdt[n-2] = (v[n-1]-v[n-3])/(t[n-1]-t[n-3])
    dvdt[n-3] = (v[n-2]-v[n-4])/(t[n-2]-t[n-4])

    # (2) Compute the rest of the points using the stencil
    if (method=='smooth'):
        # This uses a weighted average of multiple centered differences
        # (up to 3 points away) to achieve maximum noise suppression.
        c = [5./32., 4./32., 1./32.]
        for i in range(3, n-3):
            for j in range(1, 4):
                dvdt[i] += 2*j*c[j-1]*(v[i+j]-v[i-j])/(t[i+j]-t[i-j])
    elif (method == 'centered'):
        # Fallback for a standard (but still 2-point) centered difference
        for i in range(3, n-3):
            dvdt[i] = (v[i+1]-v[i-1])/(t[i+1]-t[i-1])
            
    return dvdt

In [9]:
import math

def truncated_remainder(dividend, divisor):
    """
    Computes a truncated remainder, ensuring consistent mathematical behavior
    especially for negative numbers, unlike standard Python's modulo operator.
    """
    
    # Calculate the integer part of the division (the quotient)
    # Using specific logic to ensure 'truncation' (cutting off the decimal part 
    # toward zero), which differs from standard floor/ceil for negative numbers.
    divided_number = dividend / divisor
    divided_number = -int(-divided_number) if divided_number < 0 else int(divided_number)

    # Calculate the remainder: Remainder = Dividend - Divisor * Quotient
    remainder = dividend - divisor * divided_number

    return remainder

In [10]:
import math
import numpy as np

def transform_to_pipi(input_angle):
    """
    Unwinds an angle (in radians) to ensure it changes continuously
    and does not jump from +pi to -pi (or vice versa), which is vital for
    calculating a smooth derivative (rate of change of the angle).
    """
    pi = math.pi
    
    # Calculate how many full 2*pi revolutions have occurred
    revolutions = np.floor((input_angle + np.sign(input_angle) * pi) / (2 * pi))

    # Calculate the remainder of the angle after removing full revolutions
    p1 = truncated_remainder(input_angle + np.sign(input_angle) * pi, 2 * pi)
    
    # Apply a correction term (p2) based on the sign to correctly map the angle 
    # back into the range of [-pi, pi] (this handles the unwrapping logic)
    p2 = (np.sign(np.sign(input_angle)
                  + 2 * (np.sign(math.fabs((truncated_remainder(input_angle + pi, 2 * pi))
                                          / (2 * pi))) - 1))) * pi

    # The final unwrapped angle
    output_angle = p1 - p2

    return output_angle, revolutions

In [11]:
import math
import numpy as np

def remove_acceleration_outliers(acc):
    """
    Filters out extreme acceleration spikes by replacing values above a G-force
    threshold (default 7.0G) with the previous time step's value.
    """
    
    acc_threshold_g = 7.0  # F1 cars rarely sustain forces above 5-6G
    acc = np.array(acc)
    
    # Handle the very first point
    if (math.fabs(acc[0]) > acc_threshold_g):
        acc[0] = 0.0
        
    # Iterate through the main body of the data
    for i in range(1, acc.size-1):
        if ( math.fabs(acc[i]) > acc_threshold_g ):
            # Replace the outlier with the previous, clean value
            acc[i] = acc[i-1]
            
    # Handle the very last point
    if (acc.size > 1 and math.fabs(acc[-1]) > acc_threshold_g ):
        acc[-1] = acc[-2]
            
    return acc

In [12]:
import numpy as np
import math

def compute_accelerations(physics_df):
    """
    Calculates the Longitudinal (ax) and Lateral (ay) acceleration 
    (in G's) for every point in a telemetry lap.
    """
    
    # --- 1. Longitudinal Acceleration (ax) ---
    # Convert Speed from km/h to m/s for physics calculations
    v = np.array(physics_df['Speed']) / 3.6
    
    # Calculate the rate of change of speed over time (dv/dt) using the smooth derivative
    # Then divide by 9.81 m/s^2 to convert the units from m/s^2 to G-force.
    lon_acc = smooth_derivative(physics_df['Time'], v) / 9.81 

    # --- 2. Lateral Acceleration (ay) - Requires Heading & Curvature ---
    # Calculate the derivative of X and Y positions with respect to DISTANCE (dx/ds, dy/ds)
    # These derivatives define the tangent direction (the car's path)
    dx = smooth_derivative(physics_df['Distance'], physics_df['X'])
    dy = smooth_derivative(physics_df['Distance'], physics_df['Y'])

    # Initialize the array to store the car's heading angle (theta)
    theta = np.zeros(dx.size)
    
    # Determine the initial heading angle using atan2
    if dx.size > 0 and dy.size > 0:
        theta[0] = math.atan2(dy[0], dx[0])
    else:
        # Return zeros if telemetry is empty or too short
        physics_df['Longitudinal_G'] = np.zeros_like(v)
        physics_df['Lateral_G'] = np.zeros_like(v)
        return physics_df

    # Loop to calculate the continuous heading angle (theta)
    for i in range(1, dx.size):
        # Calculate the raw change in angle from the previous point
        delta_angle = math.atan2(dy[i], dx[i]) - theta[i-1]
        
        # Use transform_to_pipi to "unwind" this change, preventing jumps from +180 to -180
        unwound_delta, _ = transform_to_pipi(delta_angle)
        
        # Update the continuous heading angle
        theta[i] = theta[i-1] + unwound_delta
        
    # Curvature (kappa) is the rate of change of the continuous heading (theta) 
    # with respect to the Distance traveled (d(theta)/ds).
    kappa = smooth_derivative(physics_df['Distance'], theta)
    
    # Lateral Acceleration (ay) is calculated using the centripetal formula:
    # ay = v^2 * kappa. This is then converted to G-force.
    lat_acc = (v * v * kappa) / 9.81
    
    # --- 3. Outlier Removal ---
    # Apply the final filtering step to remove any extreme, non-physical spikes
    physics_df['Longitudinal_G'] = remove_acceleration_outliers(lon_acc)
    physics_df['Lateral_G'] = remove_acceleration_outliers(lat_acc)
    
    return physics_df

In [ ]:
def compute_speeds(physics_df, fastest_lap):
    # Find max straight speed
    max_speed = physics_df['v_m/s'].max()
    physics_df['Max_Straight_Speed'] = max_speed

    # Find min corner speed
    # Define a "high-G corner" as the top 5% of all lateral G-forces
    HIGH_G_PERCENTILE = 95

    g_threshold = np.percentile(abs(physics_df['Lateral_G']), HIGH_G_PERCENTILE)
    high_g_corners = physics_df[abs(physics_df['Lateral_G']) >= g_threshold]
    min_corner_speed = high_g_corners['v_m/s'].min()
    physics_df['Min_Corner_Speed'] = min_corner_speed

    # Average lap speed
    total_distance_meters = physics_df['Distance'].max()
    total_time_seconds = fastest_lap['LapTime'].total_seconds()
    avg_speed_lap = total_distance_meters / total_time_seconds
    physics_df['Avg_Speed_Lap'] = avg_speed_lap

    return physics_df


In [38]:
def compute_forces(physics_df):
    # --- G-Force Features (Peak Performance) ---
    # maximum lateral G, car's maximum cornering grip
    #physics_df['LatG_max'] = max(abs(physics_df['Lateral_G']))
    physics_df['LatG_Max'] = physics_df['Lateral_G'].abs().max()

    # minimum longitudinal G, car's maximum braking force
    physics_df['LonG_Min'] = physics_df['Longitudinal_G'].min()
    # minimum longitudinal G, car's maximum acceleration
    physics_df['LonG_Max'] = physics_df['Longitudinal_G'].max()

    # --- Aero/Load Features (Setup & Car Characteristics) ---
    physics_df['DownForce_Max'] = physics_df['downForce_N'].max()
    physics_df['DownForce_Avg'] = physics_df['downForce_N'].mean()
    physics_df['DragForce_Max'] = physics_df['dragForce_N'].max()
    physics_df['LoadFront_Max'] = physics_df['loadFront_AeroModel_N'].max()
    physics_df['LoadRear_Max'] = physics_df['loadRear_AeroModel_N'].max()

    return physics_df

In [16]:
fastest_lap

Time                      0 days 01:14:52.170000
Driver                                       VER
DriverNumber                                   1
LapTime                   0 days 00:01:18.792000
LapNumber                                   17.0
Stint                                        6.0
PitOutTime                                   NaT
PitInTime                                    NaT
Sector1Time               0 days 00:00:26.262000
Sector2Time               0 days 00:00:26.483000
Sector3Time               0 days 00:00:26.047000
Sector1SessionTime        0 days 01:13:59.640000
Sector2SessionTime        0 days 01:14:26.123000
Sector3SessionTime        0 days 01:14:52.170000
SpeedI1                                    327.0
SpeedI2                                    340.0
SpeedFL                                    315.0
SpeedST                                    348.0
IsPersonalBest                              True
Compound                                    SOFT
TyreLife            

In [99]:
def compute_data_df(physics_df, fastest_lap, session, year, location):
    lap_features = pd.DataFrame([fastest_lap.to_dict()])
    
    lap_features['LapTime'] = lap_features['LapTime'].dt.total_seconds()
    lap_features['Sector1Time'] = lap_features['Sector1Time'].dt.total_seconds()
    lap_features['Sector2Time'] = lap_features['Sector2Time'].dt.total_seconds()
    lap_features['Sector3Time'] = lap_features['Sector3Time'].dt.total_seconds()
    lap_features['Location'] = location
    lap_features['Year'] = year

    lap_features['LapStartTime'] = lap_features['Time']
    weather = session.weather_data
    lap_features = pd.merge_asof(
        lap_features,                # Use the new DataFrame
        weather,
        left_on='LapStartTime', # Column from fl_df
        right_on='Time',        # Column from weather
        direction='backward'  # Finds weather row *just before* or at LapStartTime
    )
    lap_features = lap_features.rename(columns={'Time_x': 'Time'})
    lap_features = lap_features.drop(columns={'Time_y'})

    lap_features = pd.merge(lap_features, circuit_info, on='Location', how='left')
    lap_features['EventDate'] = session.event['EventDate']

    KEEP_COLS = ['Time', 'Driver', 'DriverNumber', 'Team', 'LapTime', 'LapNumber', 'Stint',
                  'Sector1Time', 'Sector2Time', 'Sector3Time', 'Compound', 'TyreLife', 'FreshTyre',
                  'AirTemp', 'TrackTemp', 'Rainfall', 'Humidity', 'WindSpeed', 'WindDirection', 
                  'TrackLenght', 'NumTurns', 'Traction', 'AsphaltGrip', 'AsphaltAbrasion', 
                  'TrackEvolution', 'TyreStress', 'Braking', 'LateralLoad', 'Downforce', 
                  'Soft', 'Medium', 'Hard', 'Location', 'Country', 'Year', 'EventDate'
                  ]
    
    lap_features = lap_features[KEEP_COLS]

    # --- Speed Features ---
    lap_features['Max_Straight_Speed'] = physics_df['v_m/s'].max()
    total_distance_meters = physics_df['Distance'].max()
    total_time_seconds = fastest_lap['LapTime'].total_seconds()
    lap_features['Avg_Lap_Speed'] = total_distance_meters / total_time_seconds

    HIGH_G_PERCENTILE = 95
    g_threshold = np.percentile(abs(physics_df['Lateral_G']), HIGH_G_PERCENTILE)
    high_g_corners = physics_df[abs(physics_df['Lateral_G']) >= g_threshold]
    lap_features['Min_Corner_Speed'] = high_g_corners['v_m/s'].min()

    # --- G-Force Features (Peak Performance) ---
    lap_features['Peak_Cornering_G'] = abs(physics_df['Lateral_G']).max()
    lap_features['Peak_Braking_G'] = physics_df['Longitudinal_G'].min()
    lap_features['Peak_Acceleration_G'] = physics_df['Longitudinal_G'].max()

    # --- Aero/Load Features (Your selected columns) ---
    lap_features['Peak_Downforce_N'] = physics_df['downForce_N'].max()
    lap_features['Avg_Downforce_N'] = physics_df['downForce_N'].mean()
    lap_features['Peak_Drag_N'] = physics_df['dragForce_N'].max()
    lap_features['Peak_Front_Load_N'] = physics_df['loadFront_AeroModel_N'].max()
    lap_features['Peak_Rear_Load_N'] = physics_df['loadRear_AeroModel_N'].max()

    return lap_features


# Building dataframe

In [20]:
YEAR = 2025
LOCATION = 'Monza'

In [100]:
fast_lap_data = fastest_lap[['Driver', 'DriverNumber']]
fast_lap_data['Year'] = YEAR
fast_lap_data['Location'] = LOCATION
fast_lap_data['EventDate'] = session.event['EventDate']

# Create a new DataFrame to calcualate data
physics_df = telemetry_driver.copy()

physics_df = calculate_physics(physics_df)

physics_df = compute_accelerations(physics_df)

# physics_df = compute_speeds(physics_df, fastest_lap)

# physics_df = compute_forces(physics_df)

lap_features = compute_data_df(physics_df, fastest_lap, session, YEAR, LOCATION)


In [103]:
lap_features

,Time,Driver,DriverNumber,Team,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,...,Avg_Lap_Speed,Min_Corner_Speed,Peak_Cornering_G,Peak_Braking_G,Peak_Acceleration_G,Peak_Downforce_N,Avg_Downforce_N,Peak_Drag_N,Peak_Front_Load_N,Peak_Rear_Load_N
0,0 days 01:14:52.170000,VER,1,Red Bull Racing,78.792,17.0,6.0,26.262,26.483,26.047,...,73.001475,52.777778,6.662891,-5.410596,1.928182,36000.640278,22274.462437,9729.902778,18120.694123,27201.10299
